# MicroController Driver Demo
This is the microcontroller driver to make a bridge between the µc and Python.  

----

In [1]:
from aad import mcd

In [2]:
print("---- List of available commands ----")
for com in mcd.CMD_LIST:
    print("*", com)

---- List of available commands ----
* CMD.SET_SR
* CMD.FILL_SRs
* CMD.SET_CS
* CMD.SET_ADR_R
* CMD.SET_ADR_C
* CMD.GET_CTL
* CMD.CLK
* CMD.CLK_SR
* CMD.CLK_XNOR
* CMD.ACK_MODE
* CMD.DEBUG_ECHO
* CMD.DEBUG_LED


----


In [3]:
print("---- List of available ports ----")
mcd.MCDriver.print_ports()

---- List of available ports ----
COM3 - Intel(R) Active Management Technology - SOL (COM3) | PID:  None
COM6 - USB Serial Device (COM6) | PID:  22336


We see that the µc is at the port `COM6`

In [4]:
driver = mcd.MCDriver()
print("Port selected by the MCDriver:", driver.ser.port)

Port selected by the MCDriver: COM6


The MCDriver selected it by default.  

---

In [5]:
driver.debug_led()

b'Hello, C2N!'

In [6]:
driver.debug_echo(b'!')

b'!'

In [7]:
# Check WLE sanity
driver.ack_mode(mcd.ACK.CLK_SR) # To make sur the clk is over before send other commands

driver.set_sr(mcd.SR.WLE, mcd.State.SET)
driver.clk_sr()
driver.set_sr(mcd.SR.WLE, mcd.State.RESET)

for _ in range(63):
	driver.clk_sr()

ctl = driver.get_ctl(mcd.SR.WLE)
print("This should give 'True':", ctl == mcd.State.SET)

# (Random result for now, it will not when the AwesomeArray will tested and working)

This should give 'True': False
